# Sumbel Ijaz, PhDEE19003 Assignment 5

# Combining All Codes

## Vgg_FC_Only

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import scikitplot as skplt
from sklearn.metrics import f1_score


In [ ]:
data_dir= 'C:/Users/840 g3/Desktop/Assignment 5 Dataset-20200421T135811Z-002/Assignment 5 Dataset'

In [ ]:
#Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(0),
                                       transforms.RandomResizedCrop(1024),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(1024),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

val_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(1024),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

#pass transform here-in
train_data = datasets.ImageFolder(root= "/content/gdrive/My Drive/Assignment_5_Dataset/train", transform=train_transforms)
test_data = datasets.ImageFolder(root= "/content/gdrive/My Drive/Assignment_5_Dataset/test", transform=test_transforms)
val_data = datasets.ImageFolder(root= "/content/gdrive/My Drive/Assignment_5_Dataset/validation", transform=val_transforms)
#train_data = datasets.ImageFolder(data_dir + '/train-set', transform=train_transforms)
#test_data = datasets.ImageFolder(data_dir + '/val-set', transform=test_transforms)

#data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True)
valloader = torch.utils.data.DataLoader(val_data, batch_size=8, shuffle=True)
                                          
print("Classes: ")
class_names = train_data.classes
print(class_names)

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(trainloader))
show_databatch(inputs, classes)

In [ ]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16(pretrained=True)
print(vgg16)
print('Output Layer of VGG16 : ', vgg16.classifier[6].out_features) # 1000 

### Removing last layers

In [ ]:
num_features = vgg16.classifier[0].in_features
features = list(vgg16.classifier.children())[:-7] # Remove last layer
print(features)

### Freezing Layers

In [ ]:
# Freeze training for all layers
for param in vgg16.features.parameters():
    param.requires_grad = False

### Adding the Layers

In [ ]:
features.extend([nn.Linear(num_features, 130)])

In [ ]:
features.extend([nn.Linear(130, 2)])

In [ ]:
print(features)

In [ ]:
vgg16.classifier = nn.Sequential(*features)
print(vgg16)

## Training 

In [ ]:
Epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.01, momentum=0.9)

In [ ]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    torch.save(vgg16.state_dict(), 'vgg16_ft.pth')

print('Finished Training')

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in trainloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Training images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Training Accuracy')
plt.xlabel('Epochs')

In [ ]:
#Confusion Matrix Part
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(trainloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Test Part

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = vgg16(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
correct = 0
total = 0
acc=[]
with torch.no_grad():
    for k in range(Epochs):
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)
        

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))

epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Test Accuracy')
plt.xlabel('Epochs')
    

In [ ]:
# Confusion Matrix
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Validation Part

In [ ]:
dataiter = iter(valloader)
images, labels = dataiter.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = vgg16(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Validation images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Validation Accuracy')
plt.xlabel('Epochs')

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(valloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

# VGG Entire

In [ ]:
# Load the pretrained model from pytorch
vgg16 = models.vgg16(pretrained=True)
print(vgg16)
print('Output Layer of VGG16 : ', vgg16.classifier[6].out_features) # 1000 

## Training Part

In [ ]:
Epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
     from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))

        
    torch.save(vgg16.state_dict(), 'vgg16_ft.pth')

print('Finished Training')
#epocharray=range(0,Epochs)
plt.plot(epoch,loss.data)
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
    from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

vgg16.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = vgg16(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))

        
    torch.save(vgg16.state_dict(), 'vgg16_ft.pth')

print('Finished Training')
#epocharray=range(0,Epochs)
plt.plot(epoch,loss.data)
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
           'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))

        
    torch.save(vgg16.state_dict(), 'vgg16_ft.pth')

print('Finished Training')
#epocharray=range(0,Epochs)
plt.plot(epoch,loss.data)
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
    

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in trainloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Training images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Training Accuracy')
plt.xlabel('Epochs')

In [ ]:
#Confusion Matrix part
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(trainloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Test Part

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = vgg16(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
correct = 0
total = 0
acc=[]
with torch.no_grad():
    for k in range(Epochs):
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)
        

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))

epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Test Accuracy')
plt.xlabel('Epochs')
    

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Validation Part

In [ ]:
dataiter = iter(valloader)
images, labels = dataiter.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = vgg16(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = vgg16(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Validation images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Validation Accuracy')
plt.xlabel('Epochs')

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(valloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = vgg16(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

# Resnet FC Only

In [ ]:
# Load the pretrained model from pytorch
resnet18 = models.resnet18(pretrained=True)
print(resnet18)
print('Output Layer of RESNET18 : ', resnet18.fc.out_features) # 1000 

## Freezing the layers 

In [ ]:
# Freeze training for all layers
for param in resnet18.layer1.parameters():
    param.requires_grad = False

## Training Part

In [ ]:
Epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.1, momentum=0.9)

In [ ]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))
        
    torch.save(resnet18.state_dict(), 'resnet18_ft.pth')

print('Finished Training')

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in trainloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Training images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Training Accuracy')
plt.xlabel('Epochs')

In [ ]:
#Confusion Matrix part
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(trainloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Testing Part

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = resnet18(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
correct = 0
total = 0
acc=[]
with torch.no_grad():
    for k in range(Epochs):
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)
        

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))

epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Test Accuracy')
plt.xlabel('Epochs')

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Validation Part

In [ ]:
dataiter1 = iter(valloader)
images, labels = dataiter1.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = resnet18(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Validation images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Validation Accuracy')
plt.xlabel('Epochs')

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(valloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

# Resnet18 Entire 

In [ ]:
Epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
     from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))

        
    torch.save(resnet18.state_dict(), 'resnet18_ft.pth')

print('Finished Training')
#epocharray=range(0,Epochs)
plt.plot(epoch,loss.data)
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
    from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet18.to(device)

resnet18.train()

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = resnet18(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))

        
    torch.save(resnet18.state_dict(), 'resnet18_ft.pth')

print('Finished Training')
#epocharray=range(0,Epochs)
plt.plot(epoch,loss.data)
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
           'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, i * len(inputs), len(trainloader.dataset),
                100. * i / len(trainloader),
                loss.data))

        
    torch.save(resnet18.state_dict(), 'resnet18_ft.pth')

print('Finished Training')
#epocharray=range(0,Epochs)
plt.plot(epoch,loss.data)
plt.ylabel('Training Loss')
plt.xlabel('Epochs')
    

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in trainloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Training images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Training Accuracy')
plt.xlabel('Epochs')

In [ ]:
#Confusion Matrix part
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(trainloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Testing Part

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = resnet18(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
correct = 0
total = 0
acc=[]
with torch.no_grad():
    for k in range(Epochs):
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)
        

print('Accuracy of the network on test images: %d %%' % (
    100 * correct / total))

epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Test Accuracy')
plt.xlabel('Epochs')

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)

## Validation Part

In [ ]:
dataiter = iter(valloader)
images, labels = dataiter.next()
show_databatch(images, labels)

In [ ]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = resnet18(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [ ]:
acc=[]
correct = 0
total = 0
with torch.no_grad():
    for k in range(Epochs):
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = resnet18(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc.append(100*correct/total)

print('Accuracy of the network on Validation images: %d %%' % (
    100 * correct / total))
epocharray=range(0,Epochs)
plt.plot(epocharray,acc)
plt.ylabel('Validation Accuracy')
plt.xlabel('Epochs')

In [ ]:
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(valloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = resnet18(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

In [ ]:
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
skplt.metrics.plot_confusion_matrix(classes, preds, figsize=(4,4), normalize=True)
plt.show()

In [ ]:
f1 = f1_score(classes, preds)
print('F1 score: %f' % f1)